In [1]:
"""Pilot experiment for computational annotator"""

'Pilot experiment for computational annotator'

In [2]:
"""some helper functions"""
import torch, pandas as pd

def get_sentences_from_dataset(path_of_dataset):
    df = pd.read_csv(path_of_dataset, delimiter="\t",header=None, 
                    names=["word", "category", "index_string", "sentence_left", "sentence_right"])
    sentences_left = df.sentence_left.tolist()
    sentences_right = df.sentence_right.tolist()
    # print(df.sample)
    word_count_left = df.sentence_left.str.split(" ").str.len()
    word_count_right = df.sentence_right.str.split(" ").str.len()
    sentence_maximum_length = max(word_count_left.max(), word_count_right.max())
    print("maximum sentence length of the dataset " + path_of_dataset.split("/")[-1] + " is: " + str(sentence_maximum_length))
    # print(df.sentence_left[word_count_left.idxmax()])
    # print(df.sentence_right[word_count_right.idxmax()])
    # print(word_count_left.idxmax())
    # print(word_count_right.idxmax())
    return sentences_left, sentences_right

from collections import defaultdict
def get_index_of_duplicates(seq):
    tally = defaultdict(list)
    for i,item in enumerate(seq):
        if (item != None):
          tally[item].append(i + 1)
    tally = dict(tally)
    # for key in tally.copy():
    #   if len(tally[key]) == 1:
    #     del tally[key]
    return tally

def get_max_sentence_length_of_a_dataset_by_tokenizer(tokenizer, path_of_dataset):
    df = pd.read_csv(path_of_dataset, delimiter="\t",header=None, 
                    names=["word", "category", "index_string", "sentence_left", "sentence_right"])
    sentences_left = df.sentence_left.tolist()
    sentences_right = df.sentence_right.tolist()

    max_len = 0
    # For every sentence...
    for sent in sentences_left + sentences_right:

        # Tokenize the text and add `[CLS]` and `[SEP]` tokens.
        input_ids = tokenizer.encode(sent, add_special_tokens=True)

        # Update the maximum sentence length.
        max_len = max(max_len, len(input_ids))
    print('Max sentence length of whole dataset by tokenizer is:', max_len)

from transformers import BertTokenizerFast
def get_BERT_tokenizer():
    # Load the BERT tokenizer.
    print('Loading BERT tokenizer...')
    tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased', do_lower_case=True)
    return tokenizer

def get_input_ids_and_so_on(tokenizer, path_of_dataset):
    df = pd.read_csv(path_of_dataset, delimiter="\t",header=None, 
                    names=["word", "category", "index_string", "sentence_left", "sentence_right"])
    sentences_left = df.sentence_left.tolist()
    sentences_right = df.sentence_right.tolist()

    # Tokenize all of the sentences and map the tokens to thier word IDs.
    input_ids_left = []
    input_ids_right = []
    attention_masks_left = []
    attention_masks_right = []
    subword_index_list_left = []
    subword_index_list_right = []

    # For every sentence...
    for sent in sentences_left:
        # `encode_plus` will:
        #   (1) Tokenize the sentence.
        #   (2) Prepend the `[CLS]` token to the start.
        #   (3) Append the `[SEP]` token to the end.
        #   (4) Map tokens to their IDs.
        #   (5) Pad or truncate the sentence to `max_length`
        #   (6) Create attention masks for [PAD] tokens.
        encoded_dict = tokenizer.encode_plus(
                            sent,                      # Sentence to encode.
                            add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                            max_length = 64,           # Pad & truncate all sentences.
                            padding='max_length',
                            return_attention_mask = True,   # Construct attn. masks.
                            return_tensors = 'pt',     # Return pytorch tensors.
                    )
        
        subword_index_list_left.append(get_index_of_duplicates(encoded_dict.word_ids()[1:-1]))

        # Add the encoded sentence to the list.    
        input_ids_left.append(encoded_dict['input_ids'])
        
        # And its attention mask (simply differentiates padding from non-padding).
        attention_masks_left.append(encoded_dict['attention_mask'])

    for sent in sentences_right:
        encoded_dict = tokenizer.encode_plus(
                            sent,                      # Sentence to encode.
                            add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                            max_length = 64,           # Pad & truncate all sentences.
                            padding='max_length',
                            return_attention_mask = True,   # Construct attn. masks.
                            return_tensors = 'pt',     # Return pytorch tensors.
                    )
        
        subword_index_list_right.append(get_index_of_duplicates(encoded_dict.word_ids()[1:-1]))
        input_ids_right.append(encoded_dict['input_ids'])
        attention_masks_right.append(encoded_dict['attention_mask'])

    # Convert the lists into tensors.
    input_ids_left = torch.cat(input_ids_left, dim=0)
    attention_masks_left = torch.cat(attention_masks_left, dim=0)

    input_ids_right = torch.cat(input_ids_right, dim=0)
    attention_masks_right = torch.cat(attention_masks_right, dim=0)

    # labels = torch.tensor(labels)

    # Print sentence 0, now as a list of IDs.
    print('sample sentence original: ', sentences_left[0])
    print('sample sentence Token IDs:', input_ids_left[0])

    return (input_ids_left, input_ids_right, attention_masks_left, attention_masks_right, subword_index_list_left, subword_index_list_right)

def get_padded_index(tokenizer, path_of_dataset, subword_index_list_left, subword_index_list_right):
    df = pd.read_csv(path_of_dataset, delimiter="\t",header=None, 
                    names=["word", "category", "index_string", "sentence_left", "sentence_right"])
    
    sentences_left = df.sentence_left.tolist()

    # print(len(subword_index_list_left))
    # print(len(subword_index_list_right))

    index_list = df.index_string
    # print(len(index_list))
    index_left_padded = list()
    index_right_padded = list()

    for index, index_string in enumerate(index_list[:]):
        word_index_left = int(index_string.split("-")[0])
        word_index_right = int(index_string.split("-")[1])
        if len(subword_index_list_left[index][word_index_left]) > 1:
            index_left_padded.append(subword_index_list_left[index][word_index_left])
        else:
            index_left_padded.append(subword_index_list_left[index][word_index_left][0])
        # if len(subword_index_list_left[index][word_index_left]) > 1:
        #   print(sentences_left[index])
        #   print(tokenizer.tokenize(sentences_left[index]))
        #   print(subword_index_list_left[index])
        if len(subword_index_list_right[index][word_index_right]) > 1:
            index_right_padded.append(subword_index_list_right[index][word_index_right])
        else:
            index_right_padded.append(subword_index_list_right[index][word_index_right][0])

        # print(len(index_left_padded))
        # print(len(index_right_padded))

    print('indices of key word in first 10 sentences:')
    print(index_list[:10])
    print('indices of key word in first 10 sentences after padding:')
    print(index_left_padded[:10])
    print(index_right_padded[:10])
    print("6th sentence: ")
    print(sentences_left[6])
    encoded_dict = tokenizer.encode_plus(
                        sentences_left[6],                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 64,           # Pad & truncate all sentences.
                        padding='max_length',
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                    )
    print('input ids of the 6th sentence:')
    print(encoded_dict['input_ids'])
    return (index_left_padded, index_right_padded)

from transformers import BertModel
import numpy as np
def save_embeddings(device, input_ids, attention_masks, index_padded, saving_path):
    input_ids = input_ids.to(device)
    attention_masks = attention_masks.to(device)
    print(input_ids.is_cuda)
    print(attention_masks.is_cuda)

    model = BertModel.from_pretrained('bert-base-uncased', output_hidden_states=True)
    model = model.to(device)

    set_length = len(input_ids)
    print(set_length)
    token_embeddings_output = list()

    layers_list = [1, 12]

    model.eval()
    for index in range(set_length):
        if index % 10 == 0:
            print(index)
        with torch.no_grad():
            outputs = model(input_ids[index].unsqueeze(0), token_type_ids=None, attention_mask=attention_masks[index].unsqueeze(0))
            hidden_states = outputs[2]
            token_embeddings = torch.stack(hidden_states, dim=0)
            # print(token_embeddings.shape)
            token_embeddings = torch.squeeze(token_embeddings, dim=1)
            # print(token_embeddings.shape)
            token_embeddings = token_embeddings.permute(1, 0, 2)
            # print(token_embeddings.shape)
            if type(index_padded[index]) is int:
                # print("enter this condition")
                sum_vec = np.sum([np.array(token_embeddings[index_padded[index]][layer].cpu()) for layer in layers_list], axis=0)
                token_embeddings_output.append(sum_vec)
            else:
                sum_vec = np.sum([np.array(token_embeddings[index_padded[index][0]][layer].cpu()) for layer in layers_list], axis=0)
                for i in range(1, len(index_padded[index])):
                    sum_vec = np.add(sum_vec, np.sum([np.array(token_embeddings[index_padded[index][i]][layer].cpu()) for layer in layers_list], axis=0))
                sum_vec = sum_vec/len(index_padded[index])
                # print(sum_vec.shape)
                token_embeddings_output.append(sum_vec)

    token_embeddings_output = np.array(token_embeddings_output)
    print(token_embeddings_output.shape)
    # print(token_embeddings_output)
    np.save(saving_path, token_embeddings_output)
    

In [3]:
'''use GPU if possible'''
# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: NVIDIA GeForce GTX 1650


In [4]:
'''prepare data for train set'''
sentences_left_train, sentences_right_train = get_sentences_from_dataset("WiC_dataset/train/train.data.txt")
tokenizer = get_BERT_tokenizer()

# # Print the original sentence.
# print(' Original: ', sentences_left_train[0])
# # Print the sentence split into tokens.
# print('Tokenized: ', tokenizer.tokenize(sentences_left_train[0]))
# # Print the sentence mapped to token ids.
# print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sentences_left_train[0])))

# print(tokenizer.tokenize('word embeddings are vectors .'))
# tokens = tokenizer('word embeddings are vectors .', add_special_tokens=True, return_attention_mask=False, return_token_type_ids=False)
# print(tokens.word_ids())
# print(get_index_of_duplicates(tokens.word_ids()[1:-1]))

get_max_sentence_length_of_a_dataset_by_tokenizer(tokenizer, "WiC_dataset/train/train.data.txt")
input_ids_left, input_ids_right, attention_masks_left, attention_masks_right, subword_index_list_left, subword_index_list_right = get_input_ids_and_so_on(tokenizer, "WiC_dataset/train/train.data.txt");
index_left_padded, index_right_padded = get_padded_index(tokenizer, "WiC_dataset/train/train.data.txt", subword_index_list_left, subword_index_list_right)

maximum sentence length of the dataset train.data.txt is: 31
Loading BERT tokenizer...
Max sentence length of whole dataset by tokenizer is: 44
sample sentence original:  You must carry your camping gear .
sample sentence Token IDs: tensor([  101,  2017,  2442,  4287,  2115, 13215,  6718,  1012,   102,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0])
indices of key word in first 10 sentences:
0    2-1
1    2-6
2    0-2
3    8-4
4    1-2
5    0-1
6    0-1
7    2-3
8    3-1
9    2-3
Name: index_string, dtype: object
indices of key word in first 10 sentences after padding:
[3, 3, 1, 9, 2, 1, [1, 2], 3, 4, 3]
[2

In [5]:
'''save embeddings for train set'''
print(input_ids_left[0].shape)
print(input_ids_left[0].unsqueeze(0).shape)
save_embeddings(device, input_ids_left, attention_masks_left, index_left_padded, 'npy_train/token_embeddings_left.npy')
save_embeddings(device, input_ids_right, attention_masks_right, index_right_padded, 'npy_train/token_embeddings_right.npy')

torch.Size([64])
torch.Size([1, 64])
True
True


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


5428
0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
1000
1010
1020
1030
1040
1050
1060
1070
1080
1090
1100
1110
1120
1130
1140
1150
1160
1170
1180
1190
1200
1210
1220
1230
1240
1250
1260
1270
1280
1290
1300
1310
1320
1330
1340
1350
1360
1370
1380
1390
1400
1410
1420
1430
1440
1450
1460
1470
1480
1490
1500
1510
1520
1530
1540
1550
1560
1570
1580
1590
1600
1610
1620
1630
1640
1650
1660
1670
1680
1690
1700
1710
1720
1730
1740
1750
1760
1770
1780
1790
1800
1810
1820
1830
1840
1850
1860
1870
1880
1890
1900
1910
1920
1930
1940
1950
1960
1970
1980
1990
2000
2010
2020
2030
2040
2050
2060
2070
2080
2090
2100
2110
2120
2130
2140
2150
2160
2170
2180
2190
2200
2

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


5428
0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
1000
1010
1020
1030
1040
1050
1060
1070
1080
1090
1100
1110
1120
1130
1140
1150
1160
1170
1180
1190
1200
1210
1220
1230
1240
1250
1260
1270
1280
1290
1300
1310
1320
1330
1340
1350
1360
1370
1380
1390
1400
1410
1420
1430
1440
1450
1460
1470
1480
1490
1500
1510
1520
1530
1540
1550
1560
1570
1580
1590
1600
1610
1620
1630
1640
1650
1660
1670
1680
1690
1700
1710
1720
1730
1740
1750
1760
1770
1780
1790
1800
1810
1820
1830
1840
1850
1860
1870
1880
1890
1900
1910
1920
1930
1940
1950
1960
1970
1980
1990
2000
2010
2020
2030
2040
2050
2060
2070
2080
2090
2100
2110
2120
2130
2140
2150
2160
2170
2180
2190
2200
2

In [6]:
'''prepare data for dev set'''
sentences_left_dev, sentences_right_dev = get_sentences_from_dataset("WiC_dataset/dev/dev.data.txt")
# tokenizer = get_BERT_tokenizer()

get_max_sentence_length_of_a_dataset_by_tokenizer(tokenizer, "WiC_dataset/dev/dev.data.txt")
input_ids_left, input_ids_right, attention_masks_left, attention_masks_right, subword_index_list_left, subword_index_list_right = get_input_ids_and_so_on(tokenizer, "WiC_dataset/dev/dev.data.txt");
index_left_padded, index_right_padded = get_padded_index(tokenizer, "WiC_dataset/dev/dev.data.txt", subword_index_list_left, subword_index_list_right)

maximum sentence length of the dataset dev.data.txt is: 29
Max sentence length of whole dataset by tokenizer is: 34
sample sentence original:  Room and board .
sample sentence Token IDs: tensor([ 101, 2282, 1998, 2604, 1012,  102,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0])
indices of key word in first 10 sentences:
0    2-2
1    0-4
2    0-1
3    1-9
4    4-6
5    3-7
6    3-1
7    4-4
8    4-8
9    7-2
Name: index_string, dtype: object
indices of key word in first 10 sentences after padding:
[3, [1, 2, 3], 1, 2, [5, 6], 4, 4, 5, 7, 8]
[3, 5, 2, 10, [10, 11], 11, 2, 5, 9, 3]
6th sentence: 
There was no meeting of minds .
input ids of the 6th sent

In [7]:
'''save embeddings for dev set'''
save_embeddings(device, input_ids_left, attention_masks_left, index_left_padded, 'npy_dev/token_embeddings_left.npy')
save_embeddings(device, input_ids_right, attention_masks_right, index_right_padded, 'npy_dev/token_embeddings_right.npy')

True
True


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


638
0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
(638, 768)
True
True


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


638
0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
(638, 768)


In [9]:
"""prepare test set"""
import numpy as np
labels_test = pd.read_csv("WiC_dataset/dev/dev.gold.txt", delimiter="\t", header=None, names=["label"])
labels_test.replace({"F": 0, "T": 1}, inplace=True)
labels_test = labels_test.values
print(len(labels_test))
embeddings_left_test = np.load('npy_dev/token_embeddings_left.npy')
print(embeddings_left_test.shape)
embeddings_right_test = np.load('npy_dev/token_embeddings_right.npy')
print(embeddings_right_test.shape)
concatenation_test = np.hstack((embeddings_left_test, embeddings_right_test))
print(concatenation_test.shape)

print(concatenation_test[0])


638
(638, 768)
(638, 768)
(638, 1536)
[ 0.98275447 -2.7962184  -0.18602687 ...  0.0239383   0.5617478
  0.9635277 ]


In [12]:
"""training process"""

embeddings_left = np.load('npy_train/token_embeddings_left.npy')
# print(embeddings_left.shape)
embeddings_right = np.load('npy_train/token_embeddings_right.npy')
# print(embeddings_right.shape)
concatenation = np.hstack((embeddings_left, embeddings_right))

labels = pd.read_csv("WiC_dataset/train/train.gold.txt", delimiter="\t", header=None, names=["label"])
labels.replace({"F": 0, "T": 1}, inplace=True)
labels = labels.values

X_train, y_train = concatenation, labels

number_samples, number_features = concatenation.shape
print("number of sample is " + str(number_samples))
print("number of feature is " + str(number_features))

X_test, y_test = concatenation_test, labels_test

from sklearn.preprocessing import StandardScaler 
sc = StandardScaler()

X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

X_train = torch.from_numpy(X_train.astype(np.float32))
X_test = torch.from_numpy(X_test.astype(np.float32))
y_train = torch.from_numpy(y_train.astype(np.float32))
y_test = torch.from_numpy(y_test.astype(np.float32))

y_train = y_train.view(y_train.shape[0], 1)
y_test = y_test.view(y_test.shape[0], 1)
print(y_train.shape)
# print(y_train)

number of sample is 5428
number of feature is 1536
torch.Size([5428, 1])


In [13]:
# 1) Model
# Linear model f = wx + b , sigmoid at the end
import torch.nn as nn

class Model(nn.Module):
    def __init__(self, n_input_features):
        super(Model, self).__init__()
        self.l1 = nn.Linear(n_input_features, 100)
        self.relu = nn.ReLU()
        self.l2 = nn.Linear(100, 1)
        # self.l1 = nn.Linear(n_input_features, 1)

    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        y_pred = torch.sigmoid(out)
        # y_pred = torch.sigmoid(self.l1(x))
        return y_pred

# 2) Loss and optimizer
num_epochs = 200
learning_rate = 0.01
criterion = nn.BCELoss()
print(number_features)
model = Model(number_features)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# 3) Training loop
for epoch in range(num_epochs):
    # Forward pass and loss
    y_pred = model(X_train)
    # print(y_pred)
    loss = criterion(y_pred, y_train)

    # Backward pass and update
    loss.backward()
    optimizer.step()

    # zero grad before new step
    optimizer.zero_grad()

    if (epoch+1) % 5 == 0:
        # print(f'epoch: {epoch+1}, loss = {loss.item():.4f}')

        with torch.no_grad():
            y_predicted = model(X_test)
            y_predicted_cls = y_predicted.round()
            acc = y_predicted_cls.eq(y_test).sum() / float(y_test.shape[0])
            print(f'epoch: {epoch+1} accuracy: {acc.item():.4f}')

1536
epoch: 5 accuracy: 0.5329
epoch: 10 accuracy: 0.5846
epoch: 15 accuracy: 0.5596
epoch: 20 accuracy: 0.5580
epoch: 25 accuracy: 0.5784
epoch: 30 accuracy: 0.5517
epoch: 35 accuracy: 0.5533
epoch: 40 accuracy: 0.5611
epoch: 45 accuracy: 0.5705
epoch: 50 accuracy: 0.5752
epoch: 55 accuracy: 0.5846
epoch: 60 accuracy: 0.5846
epoch: 65 accuracy: 0.5815
epoch: 70 accuracy: 0.5846
epoch: 75 accuracy: 0.5846
epoch: 80 accuracy: 0.5831
epoch: 85 accuracy: 0.5831
epoch: 90 accuracy: 0.5815
epoch: 95 accuracy: 0.5846
epoch: 100 accuracy: 0.5846
epoch: 105 accuracy: 0.5831
epoch: 110 accuracy: 0.5831
epoch: 115 accuracy: 0.5846
epoch: 120 accuracy: 0.5846
epoch: 125 accuracy: 0.5862
epoch: 130 accuracy: 0.5846
epoch: 135 accuracy: 0.5831
epoch: 140 accuracy: 0.5831
epoch: 145 accuracy: 0.5831
epoch: 150 accuracy: 0.5831
epoch: 155 accuracy: 0.5831
epoch: 160 accuracy: 0.5831
epoch: 165 accuracy: 0.5831
epoch: 170 accuracy: 0.5831
epoch: 175 accuracy: 0.5831
epoch: 180 accuracy: 0.5831
epoch: 